### Постановка задачи
Рассмотрим несколько моделей линейной регрессии, чтобы выяснить более оптимальную для первых 20 зданий.

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz

Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

### Подключение библиотек

In [1]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, BayesianRidge

### Загрузка данных, отсечение 20 зданий, объединение и оптимизация

In [2]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

In [3]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")
energy = energy[(energy["building_id"]<20)]
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(left=energy, right=weather, how="left",
                  left_index=True, right_index=True)
energy.reset_index(inplace=True)
energy = energy.drop(columns=["meter", "site_id", "year_built",
                              "square_feet", "floor_count"], axis=1)
del buildings
del weather
energy = reduce_mem_usage(energy)
print (energy.info())

Потребление памяти меньше на 10.39 Мб (минус 70.5 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   timestamp           175680 non-null  datetime64[ns]
 1   building_id         175680 non-null  int8          
 2   meter_reading       175680 non-null  float16       
 3   primary_use         175680 non-null  category      
 4   air_temperature     175620 non-null  float16       
 5   cloud_coverage      99080 non-null   float16       
 6   dew_temperature     175620 non-null  float16       
 7   precip_depth_1_hr   175660 non-null  float16       
 8   sea_level_pressure  173980 non-null  float16       
 9   wind_direction      170680 non-null  float16       
 10  wind_speed          175680 non-null  float16       
dtypes: category(1), datetime64[ns](1), float16(8), int8(1)
memory usage: 4.4 MB
None


### Обогащение данных: час, дни недели, праздники, логарифм

In [4]:
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
for weekday in range(0,7):
    energy['is_wday' + str(weekday)] = energy['weekday'].isin([weekday]).astype("int8")
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
energy['is_holiday'] = energy['date'].isin(us_holidays).astype("int8")
energy["meter_reading_log"] = np.log(energy["meter_reading"] + 1)

### Разделение данных

In [5]:
energy_train, energy_test = train_test_split(energy[energy["meter_reading"]>0], test_size=0.2)
print (energy_train.head())

                 timestamp  building_id  meter_reading primary_use  \
136959 2016-10-12 07:00:00           19      257.25000      Office   
174959 2016-12-30 11:00:00           19      221.12500      Office   
87471  2016-07-01 05:00:00           11      484.50000   Education   
158734 2016-11-26 16:00:00           14      454.00000   Education   
168569 2016-12-17 04:00:00            9       63.46875      Office   

        air_temperature  cloud_coverage  dew_temperature  precip_depth_1_hr  \
136959        21.703125             2.0        20.000000                0.0   
174959        10.601562             NaN        -3.900391                0.0   
87471         25.000000             NaN        23.296875                0.0   
158734        23.906250             NaN        15.000000                0.0   
168569        18.296875             0.0        15.000000                0.0   

        sea_level_pressure  wind_direction  ...  is_wday0  is_wday1  is_wday2  \
136959              101

### Линейная регрессия: по часам

In [6]:
from sklearn.metrics import r2_score

In [7]:
hours = range(0, 24)
buildings = range(0, energy_train["building_id"].max() + 1)
lr_columns = ["meter_reading_log", "hour", "building_id", "is_holiday"]
for wday in range(0,7):
    lr_columns.append("is_wday" + str(wday))
lr_models = {
    "LinearRegression": LinearRegression,
    "Lasso-0.01": Lasso,
    "Lasso-0.1": Lasso,
    "Lasso-1.0": Lasso,
    "Ridge-0.01": Ridge,
    "Ridge-0.1": Ridge,
    "Ridge-1.0": Ridge,
    "ElasticNet-1-1": ElasticNet,
    "ElasticNet-0.1-1": ElasticNet,
    "ElasticNet-1-0.1": ElasticNet,
    "ElasticNet-0.1-0.1": ElasticNet,
    "BayesianRidge": BayesianRidge
}
energy_train_lr = pd.DataFrame(energy_train, columns=lr_columns)

Линейная регрессия
\begin{equation}
z = Ax + By + C, |z-z_0|^2 \rightarrow min
\end{equation}
Лассо + LARS Лассо
\begin{equation}
\frac{1}{2n}|z-z_0|^2 + a(|A|+|B|) \rightarrow min
\end{equation}
Гребневая регрессия
\begin{equation}
|z-z_0|^2 + a(A^2 + B^2) \rightarrow min
\end{equation}
ElasticNet: Лассо + Гребневая регрессия
\begin{equation}
\frac{1}{2n}|z-z_0|^2 + \alpha p|A^2+B^2| + (\alpha - p)(|A|+|B|)/2 \rightarrow min
\end{equation}

In [8]:
lr_models_scores = {}
for _ in lr_models:
    lr_model = lr_models[_]
    energy_lr_scores = [[]]*len(buildings)
    for building in buildings:
        energy_lr_scores[building] = [0]*len(hours)
        energy_train_b = energy_train_lr[energy_train_lr["building_id"]==building]
        for hour in hours:
            energy_train_bh = energy_train_b[energy_train_b["hour"]==hour]
            y = energy_train_bh["meter_reading_log"]
            x = energy_train_bh.drop(labels=["meter_reading_log",
                "hour", "building_id"], axis=1)
            if _ in ["Ridge-0.1", "Lasso-0.1"]:
                model = lr_model(alpha=.1, fit_intercept=False).fit(x, y)
            elif _ in ["Ridge-0.01", "Lasso-0.01"]:
                model = lr_model(alpha=.01, fit_intercept=False).fit(x, y)
            elif _ == "ElasticNet-1-1":
                model = lr_model(alpha=1, l1_ratio=1, fit_intercept=False).fit(x, y)
            elif _ == "ElasticNet-1-0.1":
                model = lr_model(alpha=1, l1_ratio=.1, fit_intercept=False).fit(x, y)
            elif _ == "ElasticNet-0.1-1":
                model = lr_model(alpha=.1, l1_ratio=1, fit_intercept=False).fit(x, y)
            elif _ == "ElasticNet-0.1-0.1":
                model = lr_model(alpha=.1, l1_ratio=.05, fit_intercept=False).fit(x, y)
            else:
                model = lr_model(fit_intercept=False).fit(x, y)
            energy_lr_scores[building][hour] = r2_score(y, model.predict(x))
    lr_models_scores[_] = np.mean(energy_lr_scores)

In [9]:
print (lr_models_scores)

{'LinearRegression': 0.1336053539217353, 'Lasso-0.01': -0.19569305108000243, 'Lasso-0.1': -31.764987418282185, 'Lasso-1.0': -2483.013230734368, 'Ridge-0.01': 0.13317572160319768, 'Ridge-0.1': 0.09101491880338457, 'Ridge-1.0': -3.7897033516477063, 'ElasticNet-1-1': -2483.013230734368, 'ElasticNet-0.1-1': -31.764987418282185, 'ElasticNet-1-0.1': -2066.2678757946846, 'ElasticNet-0.1-0.1': -433.4675202627676, 'BayesianRidge': 0.13359551755117657}


### Проверим модели: LinearRegression, Lasso, BayesianRidge

In [10]:
energy_lr = [[]]*len(buildings)
energy_lasso = [[]]*len(buildings)
energy_br = [[]]*len(buildings)
for building in buildings:    
    energy_train_b = energy_train_lr[energy_train_lr["building_id"]==building]
    energy_lr[building] = [[]]*len(hours)
    energy_lasso[building] = [[]]*len(hours)
    energy_br[building] = [[]]*len(hours)
    for hour in hours:
        energy_train_bh = pd.DataFrame(energy_train_b[energy_train_b["hour"]==hour])
        y = energy_train_bh["meter_reading_log"]
        if len(y) > 0:
            x = energy_train_bh.drop(labels=["meter_reading_log",
                "hour", "building_id"], axis=1)
            model = LinearRegression(fit_intercept=False).fit(x, y)
            energy_lr[building][hour] = np.append([model.coef_], model.intercept_)
            model = Lasso(fit_intercept=False, alpha=.01).fit(x, y)
            energy_lasso[building][hour] = np.append([model.coef_], model.intercept_)
            model = BayesianRidge(fit_intercept=False).fit(x, y)
            energy_br[building][hour] = np.append([model.coef_], model.intercept_)
print (energy_lr[0][0])
print (energy_lasso[0][0])
print (energy_br[0][0])

[-0.0515393   5.43848036  5.46859976  5.47509766  5.46080633  5.51341712
  5.43010603  5.4         0.        ]
[0.         5.3671412  5.40244591 5.40343099 5.38699219 5.43863451
 5.36867746 5.314      0.        ]
[-0.05100353  5.43796645  5.46814601  5.47460551  5.46029315  5.51289999
  5.42968765  5.39941754  0.        ]
